In [1]:
import pandas as pd

# Set pandas display options to show more columns and rows
pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', 10)       # Limit to 10 rows for readability
pd.set_option('display.width', None)        # Let the display adjust to the window

# 2. Set the display width (optional but often helpful)
#    'None' tries to detect terminal width. 
#    A large number (e.g., 1000) ensures no wrapping unless absolutely necessary.
pd.set_option('display.width', 1000) 

In [2]:
# process_files.py
from config import date_str, DOWNLOAD_DIR, DEST_DIR
from pathlib import Path  # Better path handling

print(f"date_str: {date_str}")
print(f"DOWNLOAD_DIR: {DOWNLOAD_DIR}")
print(f"DEST_DIR: {DEST_DIR}\n")

# Build paths
source_path = Path(DOWNLOAD_DIR) / f'df_OHLCV_{date_str}.parquet'
dest_path = Path(DEST_DIR) / f'{date_str}_df_OHLCV_clean.parquet'

print(f"source_path: {source_path}")
print(f"dest_path: {dest_path}")

date_str: 2025-04-04
DOWNLOAD_DIR: C:\Users\ping\Downloads
DEST_DIR: ..\data

source_path: C:\Users\ping\Downloads\df_OHLCV_2025-04-04.parquet
dest_path: ..\data\2025-04-04_df_OHLCV_clean.parquet


In [3]:
# --- Data Loading & Initial Inspection ---
import pandas as pd

# # Load raw data from pickle file
# df = pd.read_pickle(source_path)

df = pd.read_parquet(source_path, engine='pyarrow')

# Display initial data structure
print("[Raw Data Overview]")
display(df.head())
df.info()

[Raw Data Overview]


Open    High     Low   Close  Adj Close   Volume  Adj Open  Adj High  Adj Low
Symbol Date                                                                                       
CLX    2025-04-04  148.72  150.84  143.37  143.86     143.86  2083767    148.72    150.84   143.37
       2025-04-03  148.46  150.60  148.04  148.69     148.69  1199500    148.46    150.60   148.04
       2025-04-02  148.05  148.34  146.05  147.62     147.62   813000    148.05    148.34   146.05
       2025-04-01  148.08  148.08  146.51  147.91     147.91  1080800    148.08    148.08   146.51
       2025-03-31  146.23  148.48  146.23  147.25     147.25  1201800    146.23    148.48   146.23

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 389718 entries, ('CLX', Timestamp('2025-04-04 00:00:00')) to ('COOP', Timestamp('2024-04-05 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       389718 non-null  float64
 1   High       389718 non-null  float64
 2   Low        389718 non-null  float64
 3   Close      389718 non-null  float64
 4   Adj Close  389718 non-null  float64
 5   Volume     389698 non-null  Int64  
 6   Adj Open   389718 non-null  float64
 7   Adj High   389718 non-null  float64
 8   Adj Low    389718 non-null  float64
dtypes: Int64(1), float64(8)
memory usage: 28.7+ MB


In [4]:
import sys
from pathlib import Path

# Notebook cell
%load_ext autoreload
%autoreload 2

# Get root directory (assuming notebook is in root/notebooks/)
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR

# Add src directory to Python path
sys.path.append(str(ROOT_DIR / 'src'))


# --- Data Filtering & Cleaning ---
import utils  # Custom utility functions

# 1. Align dates across all symbols using SPY as reference
df = utils.filter_df_dates_to_reference_symbol(df=df, reference_symbol='SPY')

# 2. Remove symbols with missing data points
df_clean, missing_symbols = utils.filter_symbols_with_missing_values(df)

# Display cleaning results
print("\n[Cleaning Report]")
print(f"Removed {len(missing_symbols)} symbols with missing data: {missing_symbols}")
print("\n[Cleaned Data Structure]")
df_clean.info()

Original number of symbols: 1560
Number of symbols after filtering: 1540
Number of symbols filtered out: 20

First 10 symbols that were filtered out:
['TEM', 'SW', 'ALTR', 'SFD', 'WAY', 'LOAR', 'VG', 'TTAN', 'ITCI', 'BTC']

Example of dates for first filtered out symbol:

Dates for TEM:
DatetimeIndex(['2025-04-04', '2025-04-03', '2025-04-02', '2025-04-01', '2025-03-31', '2025-03-28', '2025-03-27', '2025-03-26', '2025-03-25', '2025-03-24',
               ...
               '2024-06-28', '2024-06-27', '2024-06-26', '2024-06-25', '2024-06-24', '2024-06-21', '2024-06-20', '2024-06-18', '2024-06-17', '2024-06-14'], dtype='datetime64[ns]', name='Date', length=202, freq=None)

Filtered DataFrame info:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 386540 entries, ('CLX', Timestamp('2025-04-04 00:00:00')) to ('COOP', Timestamp('2024-04-05 00:00:00'))
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Open       386540 non-n

In [5]:
# --- Save Cleaned Data ---
# # Save processed data to pickle file
# df_clean.to_pickle(dest_path)

# Using PyArrow (default, recommended for most cases)
df_clean.to_parquet(dest_path, engine='pyarrow')
print(f"\n[Save Successful] Cleaned data saved to:\n{dest_path}")




[Save Successful] Cleaned data saved to:
..\data\2025-04-04_df_OHLCV_clean.parquet
